In [1]:
# Get guns data
import csv

f = open("guns.csv", "r")
data = list(csv.reader(f))
data[0:1]

[['',
  'year',
  'month',
  'intent',
  'police',
  'sex',
  'age',
  'race',
  'hispanic',
  'place',
  'education']]

In [2]:
#remove headers
headers = data.pop(0)

In [3]:
# gun death by year
years = [kill[1] for kill in data]
year_counts = {}
for kill in data:
    if kill[1] in year_counts:
        year_counts[kill[1]] += 1
    else:
        year_counts[kill[1]] = 1

In [4]:
#gun death by year and month
import datetime

dates = [datetime.datetime(year=int(kill[1]), month=int(kill[2]), day=1) for kill in data]
years = [kill[1] for kill in data]
date_counts = {}
for date in dates:
    if date in date_counts:
        date_counts[date] += 1
    else:
        date_counts[date] = 1
date_counts

{datetime.datetime(2012, 1, 1, 0, 0): 2758,
 datetime.datetime(2012, 2, 1, 0, 0): 2357,
 datetime.datetime(2012, 3, 1, 0, 0): 2743,
 datetime.datetime(2012, 4, 1, 0, 0): 2795,
 datetime.datetime(2012, 5, 1, 0, 0): 2999,
 datetime.datetime(2012, 6, 1, 0, 0): 2826,
 datetime.datetime(2012, 7, 1, 0, 0): 3026,
 datetime.datetime(2012, 8, 1, 0, 0): 2954,
 datetime.datetime(2012, 9, 1, 0, 0): 2852,
 datetime.datetime(2012, 10, 1, 0, 0): 2733,
 datetime.datetime(2012, 11, 1, 0, 0): 2729,
 datetime.datetime(2012, 12, 1, 0, 0): 2791,
 datetime.datetime(2013, 1, 1, 0, 0): 2864,
 datetime.datetime(2013, 2, 1, 0, 0): 2375,
 datetime.datetime(2013, 3, 1, 0, 0): 2862,
 datetime.datetime(2013, 4, 1, 0, 0): 2798,
 datetime.datetime(2013, 5, 1, 0, 0): 2806,
 datetime.datetime(2013, 6, 1, 0, 0): 2920,
 datetime.datetime(2013, 7, 1, 0, 0): 3079,
 datetime.datetime(2013, 8, 1, 0, 0): 2859,
 datetime.datetime(2013, 9, 1, 0, 0): 2742,
 datetime.datetime(2013, 10, 1, 0, 0): 2808,
 datetime.datetime(2013, 11,

The number of death is not significantly different if we look only by year. 
If we specify the number of death by year and month we cannot see any significant patterns.

In [5]:
#death by race and sex
def count(list, column):
    total = {}
    for row in list:
        if row[column] in total:
            total[row[column]] += 1
        else:
            total[row[column]] = 1
    return total
    
sex_counts = count(data, 5)
race_counts = count(data, 7)
race_counts

{'Asian/Pacific Islander': 1326,
 'Black': 23296,
 'Hispanic': 9022,
 'Native American/Native Alaskan': 917,
 'White': 66237}

While the number of death by date doesn't say much, we can see huge difference in terms of kills between men and women. Man are far more killed than women. 
We can also see that white men are far more killed than any other.

We can't say mush about the huge proportion of white death and we have to dig deeper in the death by race to see if this is proportionaly equivalent for each race or to see if there is some kind of "murder racism"

In [6]:
#read census
import csv

f = open("census.csv", "r")
census = list(csv.reader(f))[1]
census

['cen42010',
 'April 1, 2010 Census',
 'totsex',
 'Both Sexes',
 'tothisp',
 'Total',
 '0100000US',
 '',
 'United States',
 '308745538',
 '197318956',
 '44618105',
 '40250635',
 '3739506',
 '15159516',
 '674625',
 '6984195']

In [7]:
# race killed ratio
mapping = {
    "Asian/Pacific Islander" : int(census[14]) + int(census[15]),
    "Black" : int(census[12]), 
    "Native American/Native Alaskan" : int(census[13]),
    "Hispanic" : int(census[11]),
    "White" : int(census[10])
}

race_per_hendredk = {}

for race in race_counts:
    race_per_hendredk[race] = (race_counts[race] / mapping[race]) * 100000
race_per_hendredk

{'Asian/Pacific Islander': 8.374309664161762,
 'Black': 57.8773477735196,
 'Hispanic': 20.220491210910907,
 'Native American/Native Alaskan': 24.521955573811088,
 'White': 33.56849303419181}

In [15]:
# intent homicide
intents = [kill[3] for kill in data]
races = [kill[7] for kill in data]
homicide_race_per_hundredk = {}

for i, race in enumerate(races):
    if intents[i] == "Homicide":
        if race in homicide_race_per_hundredk:
            homicide_race_per_hundredk[race] += 1
        else:
            homicide_race_per_hundredk[race] = 1

for race in homicide_race_per_hundredk:
    homicide_race_per_hundredk[race] = (homicide_race_per_hundredk[race] / mapping[race]) * 100000

homicide_race_per_hundredk

{'Asian/Pacific Islander': 3.530346230970155,
 'Black': 48.471284987180944,
 'Hispanic': 12.627161104219914,
 'Native American/Native Alaskan': 8.717729026240365,
 'White': 4.6356417981453335}